<center> <span style="color:indigo">Machine Learning e Inferencia Bayesiana</span> </center> 

<div style="text-align: center;">
<img src="../Imagenes/CUGDL_logo.png" alt="Drawing" style="width: 800px;"/>
</div>

<center> <span style="color:DarkBlue">  Tema 6. Regresión Logística: Implementación manual con Python</span>  </center>
<center> <span style="color:Blue"> Profesor: M. en C. Iván A. Toledano Juárez </span>  </center>

# Implementación de maximum likelihood method para la regresión logística


Implementación del método por pasos, utilizando librerías estándar en Python. 

### Definir la función de entorno $L(\alpha, \beta)$

\begin{equation}
L(\beta) = \prod_{i=1}^n P_i^{y_i} (1 - P_i)^{1 - y_i}
\end{equation}

In [46]:
def likelihood(y,pi): # tiene como entradas los datos a predecir y,
    # y las probabilidades Pi
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1)) ## sumatorio de la ecuacion
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

### Calcular las probabilidades para cada observación

Las condicionales que tienen una expresión muy concreta. Hay que dar el valor de cada una de las $P_i$

\begin{equation}
P_i = P(x_i) = \frac{1}{1 + e^{-(\beta \cdot x_i)}}
\end{equation}

donde $\beta \cdot x_i = \sum_{j=0}^k \beta_j x_{ij}$ (modelo lineal).

In [47]:
def logitprobs(X,beta):
    import numpy as np ## import dentro de la función significa que numpy está disponible dentro de la función, fuera no
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi = list(range(1,n_rows + 1)) ## este vector pi tiene que tener la misma dimension
    expon = list(range(1,n_rows + 1)) ### calcular el exponente de la exponencial y luego todo lo demás
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex = X[i][j] * beta[j] # recta
            expon[i] = ex + expon[i] # exponente
        with np.errstate(divide="ignore",invalid="ignore"): ## si alguna de los calculos da error, que lo ignore
            pi[i] = 1 / (1 + np.exp(-expon[i])) ## puede dar problemas al tener un exponente grande
    return pi

### Calcular la matriz diagonal $W$

Está definida por:

\begin{equation}
W = \rm{diag} (P_i \cdot (1-P_i))_{i=1}^n
\end{equation}

In [48]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n) # matriz de ceros a la que le vaya sumando
    # se tienen n^2 ceros distribuidos en n filas y n columnas
    for i in range(n):
        print(i)
        W[i,i] = pi[i]*(1 - pi[i])
        W[i,i].astype(float)
    return W

### Definir la función logística

Aquí se utiliza el método de Newton Raphson. Se obtiene la solución de la función logística.

\begin{equation}
\beta_{n+1} = \beta_n - \frac{f(x_n)}{f^{'}(x_n)}
\end{equation}

\begin{equation}
f(\beta) = \frac{\partial l}{\partial \beta} = X(Y-P)
\end{equation}

\begin{equation}
f^{'}(\beta) = \frac{\partial^2 l}{\partial \beta^2} = XWX^{t}
\end{equation}

Tenemos que cuidar que probablemente este método no converga nunca, hay que poner un límite.

In [49]:
def logistics(X,Y, limit):
    import numpy as np
    from numpy import linalg ### para hacer la matriz inversa
    nrow = np.shape(X)[0] #numero de filas
    bias = np.ones(nrow).reshape(nrow,1) ## solo una columna, un vector de puros 1's
    X_new = np.append(X, bias, axis = 1) ## se agrega al vector original una columna mas
    ncol = np.shape(X_new)[1] ## ahora que las columnas ya fueron agregadas necesito el vector de las betas
    beta = np.zeros(ncol).reshape(ncol,1) # empezamos con ceros
    root_dif = np.array(range(1,ncol + 1)).reshape(ncol,1) # ir estimando cada una de las diferencias entre las beta
    ## no va a tener la solucion a menos que haya convergencia
    iter_i = 10000 #ir iterando mientras a un límite global, que no se pase
    while(iter_i > limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta) ## incluimos la funcion de las p_i de la informacion actual
        print("Pi:"+str(pi))
        W = findW(pi) ## se crea W con el vector de las probabilidades
        print("W:"+str(W))
        ### se tienen que llevar a cabo todas las operaciones iterativas que se mencionarion anteriormente
        # los vectores vienen en filas por numpy pero para el producto vienen en columnas
        ## por eso hay muchas transpuestas
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose()) # numerador de newthon raphson
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        ## ahora hacemos la inversa de la matriz
        ## también incremento en el numerador y denominador
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif ## aqui se hace la iteracion, la aproximación
        print('Beta:',beta)
        iter_i = np.sum(root_dif*root_dif) ## aqui se ve que el limit no es pasos, es una diferencia mínima
#        print(iter_i)
        ll = likelihood(Y,pi) # se usa la función likelihood
    return beta

### Comprobación experimental

Comprobación a mano creando un vector

In [50]:
import numpy as np

In [51]:
X = np.array(range(10)).reshape(10,1)

In [52]:
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [53]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [54]:
Y

[0, 0, 0, 0, 1, 0, 1, 0, 1, 1]

In [55]:
## añadimos más información a las X, bias
bias = np.ones(10).reshape(10,1) ## en columna
X_new = np.append(X,bias,axis=1)


In [56]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [57]:
# Comenzamos a encontrar los parámetros
a = logistics(X,Y,0.00001)

Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]
Iter:i5.777190082644626, limit:1e-05
Pi:[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), array([0.826

/var/folders/ls/3cpc9bjx04xfmf_tzhgzfgrm0000gn/T/ipykernel_14584/3955489538.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  W[i,i] = pi[i]*(1 - pi[i])


In [58]:
ll = likelihood(Y, logitprobs(X,a)) 

In [59]:
ll ## devuelve el array de informacion

array([1.32622426e-06])

El modelo devuelve la siguiente ecuación

\begin{equation}
\hat{Y} = \ln \left(\frac{P}{1-P}\right) = 0.66220827 + X *-3.69557172
\end{equation}

## Implementación con el paquete de Python `statsmodel`

In [60]:
import statsmodels.api as sm

In [61]:
logit_model = sm.Logit(Y,X_new) # 1. Instanciamos el modelo

In [62]:
result = logit_model.fit() # 2. Entrenamos el modelo

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [63]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        8
Method:                           MLE   Df Model:                            1
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                  0.3596
Time:                        12:17:41   Log-Likelihood:                -4.3101
converged:                       True   LL-Null:                       -6.7301
Covariance Type:            nonrobust   LLR p-value:                   0.02781
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6622      0.400      1.655      0.098      -0.122       1.446
const         -3.6956      2.289     -1.615      0.106      -8.182       0.791


Observamos que con el método `Logit` de `statsmodels`, nos regresa un modelo con los mismos coeficientes de nuestra implementación manual. Ahora lo intentamos con `sklearn`.

## Implementación con el paquete de Python `sklearn`

In [64]:
from sklearn.linear_model import LogisticRegression

In [65]:
lr = LogisticRegression(tol=0.00001, penalty=None, fit_intercept=False) # 1.0 Instanciamos el modelo
# Nos fijamos que nos permite marcar una tolerancia, para la convergencia
# Tenemos que especificarle si también estamos ajustando el intercept
# Incluso tenemos la opción de agregar una penalización l1 o l2

In [66]:
lr.fit(X_new, Y) # 2. Entrenamiento del modelo

LogisticRegression(fit_intercept=False, penalty=None, tol=1e-05)

In [67]:
lr.coef_

array([[ 0.66220706, -3.69557658]])

Con los parámetros adecuados, en scikit-learn también obtenemos los mismos parámetros que aquellos de nuestra implementación manual.

## Conclusión

En este notebook hemos implementado paso a paso el modelo de **regresión logística** desde cero. A través de esta práctica, pudimos observar que, más allá de ser una herramienta "mágica", este modelo se basa en conceptos claros: una combinación lineal de variables, una función sigmoide para obtener probabilidades, y una función de pérdida basada en máxima verosimilitud que se busca ser optimizada.

Este enfoque nos permite **comprender a profundidad cómo aprende el modelo**, interpretar sus coeficientes, y tener una mejor intuición al usar bibliotecas como `scikit-learn` o `statsmodels`. Además, este conocimiento es esencial para detectar errores, mejorar modelos y explicar resultados con claridad en contextos profesionales o de investigación.